In [16]:
import pandas as pd
import joblib

# Cargar el modelo previamente entrenado
model = joblib.load('car_price_model.pkl')

def validate_input(car_input):
    for column, value in car_input.items():
        if column in label_encoders and value not in label_encoders[column].classes_:
            print(f"Error: El valor '{value}' para {column} no existe en el conjunto de datos.")
            return False
    return True

# Cargar los LabelEncoders usados en el entrenamiento
label_encoders = joblib.load('label_encoders.pkl')  # Asegúrate de haber guardado estos previamente

# Crear un nuevo coche con valores sin codificar
nuevo_coche = {
    'Brand': 'Chevrolet',  # Sustituye con valores reales
    'Model': 'Impala',
    'Year': 2010,  # Año es una variable numérica
    'Fuel_Type': 'Electric',
    'Transmission': 'Automatic',
    'Mileage': 68000,
    'Engine_Size': 2.0,
    'Doors': 2,
    'Owner_Count': 1
}

if not validate_input(nuevo_coche):
    print("No se puede realizar la predicción debido a valores no válidos.")

else:
    # Convertir en DataFrame
    nuevo_coche_df = pd.DataFrame([nuevo_coche])
    
    # Aplicar Label Encoding a las columnas categóricas
    categorical_columns = ['Brand', 'Model', 'Fuel_Type', 'Transmission']
    
    for column in categorical_columns:
        if column in label_encoders:  # Verificar que el encoder existe
            nuevo_coche_df[column] = label_encoders[column].transform(nuevo_coche_df[column])
    
    # Asegurar que las columnas están en el mismo orden que el modelo espera
    nuevo_coche_df = nuevo_coche_df[model.feature_names_in_]
    
    # Realizar la predicción
    prediccion = model.predict(nuevo_coche_df)
    
    print(f'El precio estimado del coche es: {prediccion[0]}')

El precio estimado del coche es: 9834.41562470689


In [14]:
x = pd.read_csv('car_price_dataset.csv')
x

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
...,...,...,...,...,...,...,...,...,...,...
9995,Kia,Optima,2004,3.7,Diesel,Semi-Automatic,5794,2,4,8884
9996,Chevrolet,Impala,2002,1.4,Electric,Automatic,168000,2,1,6240
9997,BMW,3 Series,2010,3.0,Petrol,Automatic,86664,5,1,9866
9998,Ford,Explorer,2002,1.4,Hybrid,Automatic,225772,4,1,4084
